#통합 책 추천 시스템

- 고정 userid = 88705 / 254321 / 182459 / 161936 / 226482 / random 1 : 121184/ random 2 -> 204167
/ 제일 부정적인 사용자
- 10가지의 방식을 활용하여 user에게 추천한다.

1. 베스트셀러/작가 - 허재훈
1. 베스트셀러/지역 - 백경관
1. 콘텐츠베이스 - 책 제목 - 송도훈
1. 협력필터링 / KNN-cosine 3가지 방식 - 허재훈
1. 행렬분해 / SVD latent 150 - 허재훈
1. 행렬분해 / NMF - 송도훈
1. 행렬분해 / ALS - 허재훈

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
book_rating = pd.read_csv('/content/drive/MyDrive/bx/BX-Book-Ratings.csv',
                          sep = ';', encoding = 'latin_1', error_bad_lines = False)
book_rating.drop(index = book_rating[book_rating['Book-Rating'] == 0].index, inplace = True) #평점 0점은 결측치 처리
book_rating

<ipython-input-3-e9cc15fc3536>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  book_rating = pd.read_csv('/content/drive/MyDrive/bx/BX-Book-Ratings.csv',


,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276736,3257224281,8
7,276737,0600570967,6
...,...,...,...
1149773,276704,0806917695,5
1149775,276704,1563526298,9
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [ ]:
book_info = pd.read_csv('/content/drive/MyDrive/bx/BX-Books.csv',
                        sep = ';', encoding = 'latin_1', error_bad_lines = False)
# 불필요한 URL, 출판사 값 버리기
book_info.drop(columns = ['Image-URL-S','Image-URL-M', 'Image-URL-L', 'Publisher'], inplace = True)
book_info

<ipython-input-4-9cc67933f6d9>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  book_info = pd.read_csv('/content/drive/MyDrive/bx/BX-Books.csv',
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expec

,ISBN,Book-Title,Book-Author,Year-Of-Publication
0,0195153448,Classical Mythology,Mark P. O. Morford,2002
1,0002005018,Clara Callan,Richard Bruce Wright,2001
2,0060973129,Decision in Normandy,Carlo D'Este,1991
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999
...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988
271356,0525447644,From One to One Hundred,Teri Sloat,1991
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004
271358,0192126040,Republic (World's Classics),Plato,1996


In [ ]:
# 결측치가 매우 소수의 행에 존재하므로 결측치가 있는 행만 제거 - 도훈님 코드
book_info.dropna(inplace=True)
book_info.reset_index(drop=True, inplace=True)
book_info

,ISBN,Book-Title,Book-Author,Year-Of-Publication
0,0195153448,Classical Mythology,Mark P. O. Morford,2002
1,0002005018,Clara Callan,Richard Bruce Wright,2001
2,0060973129,Decision in Normandy,Carlo D'Este,1991
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999
...,...,...,...,...
271354,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988
271355,0525447644,From One to One Hundred,Teri Sloat,1991
271356,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004
271357,0192126040,Republic (World's Classics),Plato,1996


In [ ]:
book_user = pd.read_csv('/content/drive/MyDrive/bx/BX-Users.csv', sep = ';', encoding = 'latin_1', error_bad_lines = False)
book_user

<ipython-input-6-e195780cffae>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  book_user = pd.read_csv('/content/drive/MyDrive/bx/BX-Users.csv', sep = ';', encoding = 'latin_1', error_bad_lines = False)


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


##유저 분석

###가장 나쁜 평점을 준 랜덤유저1(총 평점갯수 대비)

In [ ]:
bad_rate_user = book_rating[book_rating['Book-Rating'] < 4] #평점 4점 이하인 유저
bad_rate_arr = bad_rate_user['User-ID'].value_counts().loc[lambda x : x >=20] #평점 갯수가 20개 이상인 유저

a = list(bad_rate_arr.index)
b = list(bad_rate_arr)
bad_rate_dict = {}

for id, count in zip(a,b): #딕셔너리 형태로 분류
    bad_rate_dict[id] = count
bad_rate_dict

{11676: 462,
 264321: 54,
 76499: 38,
 35836: 33,
 23872: 29,
 35859: 28,
 229551: 27,
 248718: 26,
 218552: 26,
 204167: 25,
 157247: 25,
 236757: 24,
 38273: 23,
 89891: 20,
 233911: 20}

In [ ]:
bad_ratio = []

for x in bad_rate_arr.index: # 위 딕셔너리의 키값과 동일
    num = book_rating[book_rating['User-ID']==x]['User-ID'].count() #해당 유저의 총 평점 갯수
    bad_ratio.append(round(bad_rate_dict[x]/num, 5)) #나쁜 평점 수 / 총 평점 수
bad_ratio # 해당 사용자의 평점 중, bad 평점의 비율

[0.0542,
 0.18947,
 0.03668,
 0.30556,
 0.05709,
 0.03378,
 0.18367,
 0.02743,
 0.1844,
 0.52083,
 0.10163,
 0.12698,
 0.07372,
 0.1,
 0.15748]

In [ ]:
bad_ratio_df = pd.DataFrame(bad_ratio, index = bad_rate_arr.index)
bad_ratio_df
# 204167 유저가 남긴 평점 대비 가장 부정적인 평점을 남긴 사람이다.

,0
11676,0.05420
264321,0.18947
76499,0.03668
35836,0.30556
23872,0.05709
35859,0.03378
229551,0.18367
248718,0.02743
218552,0.18440
204167,0.52083


In [ ]:
#204167 유저의 평점 확인
user_204167_rate = book_rating[book_rating['User-ID']==204167]
user_204167_rate.sort_values('Book-Rating',ascending = False)

,User-ID,ISBN,Book-Rating
843204,204167,067173685X,8
843203,204167,0671736809,8
843182,204167,067102020X,7
843044,204167,0373709277,7
843115,204167,0441003389,6
843234,204167,1551660717,5
843017,204167,0373034091,5
843112,204167,0440241057,5
843102,204167,0425158624,5
843146,204167,0451400933,5


### 평균 이하의 평점 갯수를 준 랜덤 유저2(사실은 한국인 중 가장 많은 평점을 준사람)

In [ ]:
korea_user_id = book_user[book_user['Location'].str.contains('korea')]['User-ID']
korea_user_id

5594        5595
6353        6354
6658        6659
7750        7751
10335      10336
           ...  
273775    273776
273790    273791
273798    273799
275278    275279
275845    275846
Name: User-ID, Length: 236, dtype: int64

In [ ]:
book_rating[book_rating['User-ID'].isin(korea_user_id)].value_counts('User-ID')[:5]

User-ID
121184    7
47465     6
156264    4
133052    4
129655    3
dtype: int64

In [ ]:
user_121184_rate = book_rating[book_rating['User-ID']==121184] #한국에서 가장 많이 평점을 준
user_121184_rate

,User-ID,ISBN,Book-Rating
501233,121184,0312995423,10
501234,121184,0399138250,9
501235,121184,0399147888,10
501236,121184,0425133516,9
501242,121184,05151324809,10
501243,121184,0515136565,8
501246,121184,0895261480,10


#SVD 방식

In [ ]:
book_rating_info = pd.merge(book_rating, book_info[['ISBN', 'Book-Title']], on = 'ISBN')
book_rating_info #book_rating에 title 추가하기

,User-ID,ISBN,Book-Rating,Book-Title
0,276726,0155061224,5,Rites of Passage
1,276729,052165615X,3,Help!: Level 1
2,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...
3,276744,038550120X,7,A Painted House
4,11676,038550120X,10,A Painted House
...,...,...,...,...
383836,276688,0425156737,2,Graven Images
383837,276688,0671563149,6,MUDDY WATER (Peter Bartholomew Mysteries)
383838,276688,1575660792,7,Gray Matter
383839,276704,0380796155,5,White Abacus


In [ ]:
# 행렬 대입전 전처리 과정

user_ids = book_rating_info["User-ID"].unique().tolist()

user2idx_dict = {x: i for i, x in enumerate(user_ids)}
idx2user_dict = {i: x for i, x in enumerate(user_ids)}

book_ids = book_rating_info["ISBN"].unique().tolist()

book2idx_dict = {x: i for i, x in enumerate(book_ids)}
idx2book_dict = {i: x for i, x in enumerate(book_ids)}

book_rating_info["user_idx"] = book_rating_info["User-ID"].map(user2idx_dict)
book_rating_info["book_idx"] = book_rating_info["ISBN"].map(book2idx_dict)

num_users = book_rating_info["User-ID"].nunique()
num_books = book_rating_info["ISBN"].nunique()
print(num_users, num_books)

book_rating_info = book_rating_info.sort_values(['user_idx', 'book_idx'])
book_rating_info = book_rating_info.reset_index(drop=True)
book_rating_info[:5]

68091 149835


,User-ID,ISBN,Book-Rating,Book-Title,user_idx,book_idx
0,276726,0155061224,5,Rites of Passage,0,0
1,276729,052165615X,3,Help!: Level 1,1,1
2,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,1,2
3,276744,038550120X,7,A Painted House,2,3
4,11676,038550120X,10,A Painted House,3,3


In [ ]:
# MinMaxScaler로 스케일링 후 값을 구했을 때, 유의미한 결과 차이를 못 느껴 그냥 생략

# from sklearn.preprocessing import MinMaxScaler

# X = book_rating_info['Book-Rating'].values.reshape(-1,1)
# X.shape

# mm_scaler = MinMaxScaler()
# mm_scaler.fit(X)

# book_rating_info['Book-Rating'] = mm_scaler.transform(X)
# book_rating_info

In [ ]:
# 희소행렬
from scipy.sparse import coo_matrix, linalg

rating_sparse_arr = coo_matrix((book_rating_info['Book-Rating'],
                                (book_rating_info['user_idx'],
                                 book_rating_info['book_idx'])),
                               shape = (num_users, num_books))
rating_sparse_arr

<68091x149835 sparse matrix of type '<class 'numpy.int64'>'
	with 383841 stored elements in COOrdinate format>

In [ ]:
rating_sparse_arr = rating_sparse_arr.astype(np.float32)

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 37.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3193686 sha256=168f7b3a6377b1a2683e13a452a766f3a7f569dd4dfbff6f417c72bc467e8e17
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [ ]:
book_rating_info.head(2) # df 컬럼 확인

,User-ID,ISBN,Book-Rating,Book-Title,user_idx,book_idx
0,276726,0155061224,5,Rites of Passage,0,0
1,276729,052165615X,3,Help!: Level 1,1,1


In [ ]:
from surprise import SVD # SVD 방식
from surprise.model_selection import GridSearchCV # 최적의 k값을 위한 GridsearchCV
from surprise import Dataset, Reader

reader = Reader(line_format='user item rating', sep = '\t')
data = Dataset.load_from_df(book_rating_info[['book_idx', 'user_idx', 'Book-Rating']], reader = reader)

param_grid = {'n_factors' : np.array([500,800,1000])} #파라미터 grid 설정(배열 속 값 중 최적의 k값을 찾는다)

#GridSearch 실행, SVD 모델로 학습하며, 측정값은 rmse 기준으로, cv는 3개로 분할.
grid_search = GridSearchCV(SVD, param_grid = param_grid, measures = ['rmse'], cv = 3)
grid_search.fit(data)

In [ ]:
best_k = grid_search.best_params['rmse']['n_factors']
best_k #최적의 k값

500

In [ ]:
#테스트를 cv 만큼 돌렸을 때의 각 평균을 보여준다.
grid_search.cv_results['mean_test_rmse']

array([3.20775351, 3.20777279, 3.20775582])

In [ ]:
#최적의 k 값을 가지고 SVD
U, S, V = linalg.svds(rating_sparse_arr, k = best_k)

In [ ]:
U.shape, S.shape, V.shape

((68091, 500), (500,), (500, 149835))

##타겟 유저 설정

In [ ]:
#타겟 유저 설정(User-ID 값 입력)
svd_target_user_id = 161936

In [ ]:
#User-ID값을 book_rating_info의 user_idx로 변환시켜준다.
svd_target_user_df = book_rating_info[book_rating_info['User-ID'] == svd_target_user_id]
svd_target_user_idx = svd_target_user_df.user_idx.iloc[0]
svd_target_user_idx

5623

In [ ]:
#타겟 유저의 예측값 찾아내기
pred_target_user_item_rating = U[svd_target_user_idx] @ np.diag(S) @ V

In [ ]:
print(pred_target_user_item_rating.shape)
print(pred_target_user_item_rating) #타겟 유저의 책 별 예측값
print(rating_sparse_arr.shape)

(149835,)
[-1.44227712e-07 -1.09675591e-06  1.12887555e-07 ... -2.36757123e-03
  7.90650811e-05  7.90650811e-05]
(68091, 149835)


In [ ]:
#book_idx로 묶기(다중값은 평균값으로 rating을 받아올 것)
book_rating_avg_by_book_idx = book_rating_info.groupby('book_idx').agg('mean')
#불필요한 칼럼 제거
book_rating_avg_by_book_idx.drop(columns = ['User-ID', 'user_idx'],inplace = True)
#타겟 예측 값을 칼럼에 추가
book_rating_avg_by_book_idx['pred_rate'] = pred_target_user_item_rating
#예측 rating이 높은 순으로 정리
book_rating_avg_by_book_idx = book_rating_avg_by_book_idx.sort_values('pred_rate', ascending = False)

In [ ]:
#데이터 확인
book_rating_avg_by_book_idx

,Book-Rating,pred_rate
book_idx,,
468,7.982014,10.065306
2305,8.000000,9.100590
5724,8.056000,8.627345
3102,8.318182,6.422138
7411,7.450000,6.405055
...,...,...
2719,8.213483,-0.735878
13855,7.578947,-0.788972
9231,7.688679,-0.927307


In [ ]:
# 유저가 읽은 'Book-Title' 설정
seen_book = list(set(svd_target_user_df['Book-Title'].to_list()))

In [ ]:
rec_index = list(book_rating_avg_by_book_idx.index) #추천하는 책의 인덱스
count = 0
target_unseen_rec_list = [] # 타겟 유저가 읽지 않은 책의 인덱스를 담을 리스트

for idx in rec_index:
    title = list(book_rating_info[book_rating_info['book_idx'] == idx]['Book-Title'])[0] #idx의 제목 반환
    if count >= 5: #추천은 5권까지만 받자.
        break
    if title not in seen_book: # 안본 책 체크
        target_unseen_rec_list.append(idx)
        count += 1
target_unseen_rec_list # book_idx 값이다.

[1167, 5452, 9154, 24, 819]

##결론 도출

In [ ]:
#타겟 유저 다시 확인
svd_target_user_id

161936

In [ ]:
#TOP 1
top1_rec_df = book_rating_info[book_rating_info['book_idx']==target_unseen_rec_list[0]]
top1_rec_df = top1_rec_df[['ISBN','Book-Title']]
top1_rec_df.head(1) #보기편하게 head로 처리

,ISBN,Book-Title
266,1558745157,The Lost Boy: A Foster Child's Search for the ...


In [ ]:
#TOP 2
top2_rec_df = book_rating_info[book_rating_info['book_idx']==target_unseen_rec_list[1]]
top2_rec_df = top2_rec_df[['ISBN','Book-Title']]
top2_rec_df.head(1)#보기편하게 head로 처리

,ISBN,Book-Title
892,1573225789,The Color of Water: A Black Man's Tribute to H...


In [ ]:
#TOP 3
top3_rec_df = book_rating_info[book_rating_info['book_idx']==target_unseen_rec_list[2]]
top3_rec_df = top3_rec_df[['ISBN','Book-Title']]
top3_rec_df.head(1)#보기편하게 head로 처리

,ISBN,Book-Title
1346,0452281903,A Man Named Dave: A Story of Triumph and Forgi...


In [ ]:
#TOP 4
top4_rec_df = book_rating_info[book_rating_info['book_idx']==target_unseen_rec_list[3]]
top4_rec_df = top4_rec_df[['ISBN','Book-Title']]
top4_rec_df.head(1)#보기편하게 head로 처리

,ISBN,Book-Title
13603,0330332775,Bridget Jones's Diary
